# Data Splits
We want to split the SynFerm data set into train, validation and test data.
For all splits, we will do 9 random repetitions.
For 1D and 2D split, which both use 3 different groups to split on, these will divide into 3 random repetitions for each of the 3 groups. 

### 0D Split
For the 0D split, we use a random train-test split.
We use a 80/10/10 split into train, val, and test set.

### 1D Split
For the 1D split, we use a (1D) GroupShuffleSplit.
Each individual split will be 80/10/10 train/test (of groups not samples!).
As groups, we use either initiator, monomer, or terminator.

### 2D Split
For the 2D split, we use a (2D) GroupShuffleSplit.
Each individual split will use 10% of groups as test set and 12.5% of remaining groups as validation set. 
Due to the dimensionality, this means we expect 0.1 * 0.1 = 1% of samples in the test and validation set and 0.900^2 * 0.875^2 = 62.0% of sample in the training set.
The remaining samples are not used to prevent leakage.
As groups, we use either \[initiator, monomer], \[monomer, terminator] or \[initiator, terminator].

### 3D Split
For the 3D split, we use a (3D) GroupShuffleSplit.
Each individual split will use 20% of groups as test set, 25% of remaining groups as validation set, and the remaining groups as training set.
Due to the dimensionality, this means we expect 0.2^3 = 0.8% of samples in the test and validation set and 0.800^3 * 0.750^3 = 21.6% of sample in the training set.

In [ ]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

from src.definitions import DATA_DIR
from src.util.train_test_split import GroupShuffleSplitND
from util import write_indices_and_stats

In [ ]:
# Load data
data_filename = "synferm_dataset_2023-09-05_40018records.csv"
data_name = data_filename.rsplit("_", maxsplit=1)[0]
df = pd.read_csv(DATA_DIR / "curated_data" / data_filename)
df.shape

In [ ]:
df.head()

## 0D split

In [ ]:
splitter = ShuffleSplit(n_splits=9, test_size=0.1, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.1/0.9, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [ ]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=0, 
    save_indices=True, 
    train_size=""
)

## 1D split

In [ ]:
splitter = GroupShuffleSplit(n_splits=3, test_size=0.1, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.1/0.9, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [ ]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["M_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["M_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
        
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["T_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["T_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    train_size=""
)

## 2D split

In [ ]:
splitter = GroupShuffleSplitND(n_splits=3, test_size=0.1, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.1/0.9, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [ ]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df, groups=df[["I_long", "M_long"]]):
    train, val = next(inner_splitter.split(df.iloc[idx_train_val], groups=df[["I_long", "M_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df[["M_long", "T_long"]]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df[["M_long", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df[["I_long", "T_long"]]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df[["I_long", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
        
print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    train_size=""
)

## 3D split

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.2, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState (not true, copyPaste error from before. Anyway, not a problem)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.2/0.8, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [ ]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df, groups=df[["I_long", "M_long", "T_long"]]):
    train, val = next(inner_splitter.split(df.iloc[idx_train_val], groups=df[["I_long", "M_long", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    train_size=""
)

## Control: Show statistics for previously prepared splits

In [ ]:
# Load data
data_filename = "synferm_dataset_2023-09-05_40018records.csv"
data_name = data_filename.rsplit("_", maxsplit=1)[0]
df = pd.read_csv(DATA_DIR / "curated_data" / data_filename)
df.shape

### 0D split

In [ ]:
split_dimension = 0
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9):
    with open(split_dir / f"fold{fold_idx}_statistics_multiclass.txt", "w") as f:
        # import indices
        train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
        val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
        test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()
        
        # check mutually exclusive
        assert len(np.intersect1d(train_idx, val_idx)) == 0
        assert len(np.intersect1d(train_idx, test_idx)) == 0
        assert len(np.intersect1d(val_idx, test_idx)) == 0
        
        # determine multiclass distribution
        train_dist = df["major_A-C"].iloc[train_idx].value_counts().sort_index()
        val_dist = df["major_A-C"].iloc[val_idx].value_counts().sort_index()
        test_dist = df["major_A-C"].iloc[test_idx].value_counts().sort_index()

        for i, item in (train_dist / len(train_idx)).items():
            f.write(f"Training set 'major_A-C' class {i}: {item:.1%}\n")
        for i, item in (val_dist / len(val_idx)).items():
            f.write(f"Validation set 'major_A-C' class {i}: {item:.1%}\n")
        for i, item in (test_dist / len(test_idx)).items():
            f.write(f"Test set 'major_A-C' class {i}: {item:.1%}\n")


### 1D split

In [ ]:
split_dimension = 1
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9):
    with open(split_dir / f"fold{fold_idx}_statistics_multiclass.txt", "w") as f:
        # import indices
        train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
        val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
        test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()
        
        # check mutually exclusive
        assert len(np.intersect1d(train_idx, val_idx)) == 0
        assert len(np.intersect1d(train_idx, test_idx)) == 0
        assert len(np.intersect1d(val_idx, test_idx)) == 0
        
        # check 1D groups are mutually exclusive
        if fold_idx < 3: # first three are split on initiator
            assert len(np.intersect1d(df["I_long"].iloc[train_idx], df["I_long"].iloc[val_idx])) == 0
            assert len(np.intersect1d(df["I_long"].iloc[train_idx], df["I_long"].iloc[test_idx])) == 0
            assert len(np.intersect1d(df["I_long"].iloc[val_idx], df["I_long"].iloc[test_idx])) == 0
        elif fold_idx < 6:  # next three are split on monomer
            assert len(np.intersect1d(df["M_long"].iloc[train_idx], df["M_long"].iloc[val_idx])) == 0
            assert len(np.intersect1d(df["M_long"].iloc[train_idx], df["M_long"].iloc[test_idx])) == 0
            assert len(np.intersect1d(df["M_long"].iloc[val_idx], df["M_long"].iloc[test_idx])) == 0
        else:  # last three are split on terminator
            assert len(np.intersect1d(df["T_long"].iloc[train_idx], df["T_long"].iloc[val_idx])) == 0
            assert len(np.intersect1d(df["T_long"].iloc[train_idx], df["T_long"].iloc[test_idx])) == 0
            assert len(np.intersect1d(df["T_long"].iloc[val_idx], df["T_long"].iloc[test_idx])) == 0
        
        # determine multiclass distribution
        train_dist = df["major_A-C"].iloc[train_idx].value_counts().sort_index()
        val_dist = df["major_A-C"].iloc[val_idx].value_counts().sort_index()
        test_dist = df["major_A-C"].iloc[test_idx].value_counts().sort_index()

        for i, item in (train_dist / len(train_idx)).items():
            f.write(f"Training set 'major_A-C' class {i}: {item:.1%}\n")
        for i, item in (val_dist / len(val_idx)).items():
            f.write(f"Validation set 'major_A-C' class {i}: {item:.1%}\n")
        for i, item in (test_dist / len(test_idx)).items():
            f.write(f"Test set 'major_A-C' class {i}: {item:.1%}\n")


### 2D split

In [ ]:
split_dimension = 2
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9):
    with open(split_dir / f"fold{fold_idx}_statistics_multiclass.txt", "w") as f:
        # import indices
        train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
        val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
        test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()
        
        # check mutually exclusive
        assert len(np.intersect1d(train_idx, val_idx)) == 0
        assert len(np.intersect1d(train_idx, test_idx)) == 0
        assert len(np.intersect1d(val_idx, test_idx)) == 0
        
        # check 2D groups are mutually exclusive
        if fold_idx < 3: # first three are split on initiator and monomer
            assert len(np.intersect1d(np.unique(df[["I_long", "M_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long"]].iloc[val_idx]))) == 0
            assert len(np.intersect1d(np.unique(df[["I_long", "M_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long"]].iloc[test_idx]))) == 0
            assert len(np.intersect1d(np.unique(df[["I_long", "M_long"]].iloc[val_idx]), np.unique(df[["I_long", "M_long"]].iloc[test_idx]))) == 0
        elif fold_idx < 6:  # next three are split on monomer and terminator
            assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[train_idx]), np.unique(df[["M_long", "T_long"]].iloc[val_idx]))) == 0
            assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[train_idx]), np.unique(df[["M_long", "T_long"]].iloc[test_idx]))) == 0
            assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[val_idx]), np.unique(df[["M_long", "T_long"]].iloc[test_idx]))) == 0
        else:  # last three are split on initiator and terminator
            assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "T_long"]].iloc[val_idx]))) == 0
            assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "T_long"]].iloc[test_idx]))) == 0
            assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[val_idx]), np.unique(df[["I_long", "T_long"]].iloc[test_idx]))) == 0
            
        # determine multiclass distribution
        train_dist = df["major_A-C"].iloc[train_idx].value_counts().sort_index()
        val_dist = df["major_A-C"].iloc[val_idx].value_counts().sort_index()
        test_dist = df["major_A-C"].iloc[test_idx].value_counts().sort_index()

        for i, item in (train_dist / len(train_idx)).items():
            f.write(f"Training set 'major_A-C' class {i}: {item:.1%}\n")
        for i, item in (val_dist / len(val_idx)).items():
            f.write(f"Validation set 'major_A-C' class {i}: {item:.1%}\n")
        for i, item in (test_dist / len(test_idx)).items():
            f.write(f"Test set 'major_A-C' class {i}: {item:.1%}\n")


### 3D split

In [ ]:
split_dimension = 3
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9):
    with open(split_dir / f"fold{fold_idx}_statistics_multiclass.txt", "w") as f:
        # import indices
        train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
        val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
        test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()
        
        # check mutually exclusive
        assert len(np.intersect1d(train_idx, val_idx)) == 0
        assert len(np.intersect1d(train_idx, test_idx)) == 0
        assert len(np.intersect1d(val_idx, test_idx)) == 0
        
        # check 3D groups are mutually exclusive
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[val_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[test_idx]))) == 0
            
        # determine multiclass distribution
        train_dist = df["major_A-C"].iloc[train_idx].value_counts().sort_index()
        val_dist = df["major_A-C"].iloc[val_idx].value_counts().sort_index()
        test_dist = df["major_A-C"].iloc[test_idx].value_counts().sort_index()

        for i, item in (train_dist / len(train_idx)).items():
            f.write(f"Training set 'major_A-C' class {i}: {item:.1%}\n")
        for i, item in (val_dist / len(val_idx)).items():
            f.write(f"Validation set 'major_A-C' class {i}: {item:.1%}\n")
        for i, item in (test_dist / len(test_idx)).items():
            f.write(f"Test set 'major_A-C' class {i}: {item:.1%}\n")